In [2]:
cwd = "/home/zzz0054/chen_data"
reads = ["C0TR-1_S1_R1_001", "C0TR1aux_S1_R1_001"]
subF = "Alonso"

In [2]:
!mkdir -p {cwd}/trimmed_out/
!mkdir -p {cwd}/before_trim/
!mkdir -p {cwd}/quality_filtered
!mkdir -p {cwd}/after_trim/
!mkdir -p  {cwd}/aligned

In [12]:
!mkdir -p {cwd}/trimmed_out/
for read in reads:
    # -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -e 0.2 -m 80 -M 180
    !cutadapt -a CTGTAGGCACCATCAAT -e 0.2 -m 15 -M 60 -o {cwd}/trimmed_out/{read}.trimmed.fastq {cwd}/{subF}/{read}.fastq > {cwd}/trimmed_out/{read}_trimmed.log

Done           00:01:13     7,796,718 reads @   9.4 µs/read;   6.37 M reads/minute
Done           00:07:52    49,092,029 reads @   9.6 µs/read;   6.24 M reads/minute


In [13]:
# quality check
!mkdir {cwd}/before_trim/
for read in reads:
    !fastqc {cwd}/{subF}/{read}.fastq -o {cwd}/before_trim/ -a {cwd}/adapter.txt

mkdir: cannot create directory ‘/home/zzz0054/chen_data/before_trim/’: File exists
null
Started analysis of C0TR-1_S1_R1_001.fastq
Approx 5% complete for C0TR-1_S1_R1_001.fastq
Approx 10% complete for C0TR-1_S1_R1_001.fastq
Approx 15% complete for C0TR-1_S1_R1_001.fastq
Approx 20% complete for C0TR-1_S1_R1_001.fastq
Approx 25% complete for C0TR-1_S1_R1_001.fastq
Approx 30% complete for C0TR-1_S1_R1_001.fastq
Approx 35% complete for C0TR-1_S1_R1_001.fastq
Approx 40% complete for C0TR-1_S1_R1_001.fastq
Approx 45% complete for C0TR-1_S1_R1_001.fastq
Approx 50% complete for C0TR-1_S1_R1_001.fastq
Approx 55% complete for C0TR-1_S1_R1_001.fastq
Approx 60% complete for C0TR-1_S1_R1_001.fastq
Approx 65% complete for C0TR-1_S1_R1_001.fastq
Approx 70% complete for C0TR-1_S1_R1_001.fastq
Approx 75% complete for C0TR-1_S1_R1_001.fastq
Approx 80% complete for C0TR-1_S1_R1_001.fastq
Approx 85% complete for C0TR-1_S1_R1_001.fastq
Approx 90% complete for C0TR-1_S1_R1_001.fastq
Approx 95% complete for 

In [14]:
!mkdir -p {cwd}/quality_filtered
for read in reads:
    !fastq_quality_filter -Q33 -v -q 25 -p 75 -i {cwd}/trimmed_out/{read}.trimmed.fastq -o {cwd}/quality_filtered/{read}.trimmed.Qfilter.fastq > {cwd}/quality_filtered/{read}.Qfilter.log

In [15]:
# quality check
!mkdir {cwd}/after_trim/
for read in reads:
    !fastqc {cwd}/quality_filtered/{read}.trimmed.Qfilter.fastq -o {cwd}/after_trim/ -a {cwd}/adapter.txt

mkdir: cannot create directory ‘/home/zzz0054/chen_data/after_trim/’: File exists
null
Started analysis of C0TR-1_S1_R1_001.trimmed.Qfilter.fastq
Approx 5% complete for C0TR-1_S1_R1_001.trimmed.Qfilter.fastq
Approx 10% complete for C0TR-1_S1_R1_001.trimmed.Qfilter.fastq
Approx 15% complete for C0TR-1_S1_R1_001.trimmed.Qfilter.fastq
Approx 20% complete for C0TR-1_S1_R1_001.trimmed.Qfilter.fastq
Approx 25% complete for C0TR-1_S1_R1_001.trimmed.Qfilter.fastq
Approx 30% complete for C0TR-1_S1_R1_001.trimmed.Qfilter.fastq
Approx 35% complete for C0TR-1_S1_R1_001.trimmed.Qfilter.fastq
Approx 40% complete for C0TR-1_S1_R1_001.trimmed.Qfilter.fastq
Approx 45% complete for C0TR-1_S1_R1_001.trimmed.Qfilter.fastq
Approx 50% complete for C0TR-1_S1_R1_001.trimmed.Qfilter.fastq
Approx 55% complete for C0TR-1_S1_R1_001.trimmed.Qfilter.fastq
Approx 60% complete for C0TR-1_S1_R1_001.trimmed.Qfilter.fastq
Approx 65% complete for C0TR-1_S1_R1_001.trimmed.Qfilter.fastq
Approx 70% complete for C0TR-1_S1_R1

In [17]:
!mkdir -p  {cwd}/aligned
for read in reads:
    !bowtie --chunkmbs 200 -n 2 -y -a --norc --best --strata -S -p 16 -l 25 --un={cwd}/aligned/noncontam_{read}.fastq {cwd}/bowtie_index/bowtie_index -q {cwd}/quality_filtered/{read}.trimmed.Qfilter.fastq {cwd}/aligned/{read}.alin

Setting the index via positional argument will be deprecated in a future release. Please use -x option instead.
# reads processed: 7507557
# reads with at least one alignment: 1089542 (14.51%)
# reads that failed to align: 6418015 (85.49%)
Reported 3933710 alignments
Setting the index via positional argument will be deprecated in a future release. Please use -x option instead.
# reads processed: 46692636
# reads with at least one alignment: 4689323 (10.04%)
# reads that failed to align: 42003313 (89.96%)
Reported 16639326 alignments


In [6]:
for read in reads:
    !samtools view -bS {cwd}/aligned/{read}.alin> {cwd}/aligned/{read}.bam

--runThreadN 32: Specifies the number of threads to use for parallel execution. In this case, it uses 32 threads.
--outFilterType Normal: Sets the type of filtering to apply to the alignments. "Normal" means it will use the default filtering criteria.
--outWigType wiggle: Specifies the output format for the signal coverage. "wiggle" generates a wiggle file for visualization.
--outWigStrand Stranded: Indicates that the output wiggle file should be stranded, meaning it will have separate tracks for forward and reverse strands.
--outWigNorm RPM: Sets the normalization method for the wiggle output. "RPM" means reads per million, which normalizes the signal based on the total number of mapped reads.
--alignEndsType EndToEnd: Specifies the alignment type. "EndToEnd" means the entire read must align from one end to the other without any clipping.
--outFilterMismatchNmax 1: Sets the maximum number of mismatches allowed per read. In this case, reads with more than 1 mismatch will be filtered out.
--outFilterMultimapNmax 1: Sets the maximum number of alignments allowed per read. Reads that align to more than one location will be filtered out.
--genomeDir {cwd}/STAR_genome_index/: Specifies the path to the directory containing the STAR genome index files.
--readFilesIn {cwd}/aligned/noncontam_{read}.fastq: Specifies the input FASTQ file(s) containing the reads to be aligned. {read} is likely a placeholder for the read identifier (e.g., R1, R2).
--outFileNamePrefix {cwd}/star_out/{read}/: Sets the prefix for the output files. The output files will be written to the star_out/{read}/ directory.
--outSAMtype BAM SortedByCoordinate: Specifies the output format for the alignments. In this case, it generates a sorted BAM file.
--quantMode TranscriptomeSAM GeneCounts: Enables quantification of reads per gene. It generates a transcriptome alignment file (in SAM format) and a table with gene-level read counts.
--outSAMattributes All: Specifies the attributes to include in the output SAM/BAM file. "All" means it will include all available attributes.

In [4]:
# for read in reads:
for read in ['C0TR-1_S1_R1_001']:
    !STAR --genomeDir {cwd}/STAR_genome_index/ --runThreadN 64 --readFilesIn {cwd}/aligned/noncontam_{read}.fastq --outFilterMultimapNmax 20 --outFilterScoreMinOverLread 0 --outFilterMatchNminOverLread 0 --outFilterMismatchNmax 2 --outFileNamePrefix {cwd}/star_out/{read}/ --outSAMtype BAM SortedByCoordinate --outSAMunmapped Within --outSAMattributes Standard --quantMode TranscriptomeSAM GeneCounts
    
    # at least

	/home/zzz0054/miniconda3/envs/ribo4/bin/STAR-avx2 --genomeDir /home/zzz0054/chen_data/STAR_genome_index/ --runThreadN 64 --readFilesIn /home/zzz0054/chen_data/aligned/noncontam_C0TR-1_S1_R1_001.fastq --outFilterMultimapNmax 20 --outFilterScoreMinOverLread 0 --outFilterMatchNminOverLread 0 --outFilterMismatchNmax 2 --outFileNamePrefix /home/zzz0054/chen_data/star_out/C0TR-1_S1_R1_001/ --outSAMtype BAM SortedByCoordinate --outSAMunmapped Within --outSAMattributes Standard --quantMode TranscriptomeSAM GeneCounts
	STAR version: 2.7.11b   compiled: 2024-03-19T08:38:59+0000 :/opt/conda/conda-bld/star_1710837244939/work/source
Jul 24 08:36:01 ..... started STAR run
Jul 24 08:36:01 ..... loading genome
Jul 24 08:36:04 ..... started mapping
Jul 24 08:36:37 ..... finished mapping
Jul 24 08:36:37 ..... started sorting BAM
Jul 24 08:36:42 ..... finished successfully


In [7]:
!samtools view -c /home/zzz0054/chen_data/star_out/C0TR-1_S1_R1_001/Aligned.toTranscriptome.out.bam

222679


In [10]:
for read in reads:
    !STAR --runThreadN 32 --outFilterType Normal --outWigType wiggle --outWigStrand Stranded --outWigNorm RPM --alignEndsType EndToEnd --outFilterMismatchNmax 1 --outFilterMultimapNmax 1 --genomeDir {cwd}/STAR_genome_index/ --readFilesIn {cwd}/aligned/noncontam_{read}.fastq --outFileNamePrefix  {cwd}/star_out/{read}/ --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts --outSAMattributes All

	/home/zzz0054/miniconda3/envs/ribo4/bin/STAR-avx2 --runThreadN 32 --outFilterType Normal --outWigType wiggle --outWigStrand Stranded --outWigNorm RPM --alignEndsType EndToEnd --outFilterMismatchNmax 1 --outFilterMultimapNmax 1 --genomeDir /home/zzz0054/chen_data/STAR_genome_index/ --readFilesIn /home/zzz0054/chen_data/aligned/noncontam_C0TR-1_S1_R1_001.fastq --outFileNamePrefix /home/zzz0054/chen_data/star_out/C0TR-1_S1_R1_001/ --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts --outSAMattributes All
	STAR version: 2.7.11b   compiled: 2024-03-19T08:38:59+0000 :/opt/conda/conda-bld/star_1710837244939/work/source
Jul 22 18:39:19 ..... started STAR run
Jul 22 18:39:19 ..... loading genome
Jul 22 18:39:22 ..... started mapping
Jul 22 18:39:52 ..... finished mapping
Jul 22 18:39:52 ..... started sorting BAM
Jul 22 18:39:52 ..... started wiggle output
Jul 22 18:39:57 ..... finished successfully
	/home/zzz0054/miniconda3/envs/ribo4/bin/STAR-avx2 --runThreadN 32 --out

In [3]:
for read in reads:
    !samtools sort -T {cwd}/star_out/{read}/Aligned.toTranscriptome.out.sorted -o {cwd}/star_out/{read}/Aligned.toTranscriptome.out.sorted.bam {cwd}/star_out/{read}/Aligned.toTranscriptome.out.bam
    !samtools index {cwd}/star_out/{read}/Aligned.toTranscriptome.out.sorted.bam ## mapped to transcriptome
    !samtools index {cwd}/star_out/{read}/Aligned.sortedByCoord.out.bam ## mapped to genome
    
    # !samtools sort -T {cwd}/aligned/{read}.bam_sort -o {cwd}/aligned/{read}.sort.bam {cwd}/aligned/{read}.bam
    # !samtools index {cwd}/aligned/{read}.sort.bam ## mapped to transcriptome


[bam_sort_core] merging from 1 files and 1 in-memory blocks...


In [4]:
# !samtools merge -X {cwd}/aligned/out.bam {cwd}/aligned/{reads[0]}.sort.bam {cwd}/aligned/{reads[1]}.sort.bam
!samtools merge -f -X {cwd}/aligned/out.bam {cwd}/star_out/{reads[0]}/Aligned.toTranscriptome.out.sorted.bam {cwd}/star_out/{reads[1]}/Aligned.toTranscriptome.out.sorted.bam

In [5]:
# for read in reads:
!mkdir -p {cwd}/metaplots/{read}/
!metaplots -a {cwd}/ref/annot -r {cwd}/aligned/out.bam -o {cwd}/metaplots/

[2024-07-23 20:26:20] Create metaplot file and predict the P-site locations ...
	Loading transcripts.pickle ...
[E::idx_find_and_load] Could not retrieve index file for '/home/zzz0054/chen_data/aligned/out.bam'
[2024-07-23 20:26:28] Complete prediction of the P-site locations
